In [4]:
# %pip install duckdb

In [5]:
import duckdb

1. Create connection and a cursor

In [6]:
conn= duckdb.connect("database.db")

In [7]:
conn

Cursor is a way to connect a query and the results. Ask the cursor to execute SQL, and use it to get te results back. 

In [8]:
cur = conn.cursor()

Now lets do something with our cursor:

In [9]:
cur.execute("SELECT * FROM Site LIMIT 5")

This returns a list 
- each row returns a tuple of values 

In [10]:
cur.fetchall()

[('barr',
  'Barrow',
  'Alaska, USA',
  71.30000305175781,
  -156.60000610351562,
  220.39999389648438),
 ('burn',
  'Burntpoint Creek',
  'Ontario, Canada',
  55.20000076293945,
  -84.30000305175781,
  63.0),
 ('bylo',
  'Bylot Island',
  'Nunavut, Canada',
  73.19999694824219,
  -80.0,
  723.5999755859375),
 ('cakr',
  'Cape Krusenstern',
  'Alaska, USA',
  67.0999984741211,
  -163.5,
  54.099998474121094),
 ('cari',
  'Canning River Delta',
  'Alaska, USA',
  70.0999984741211,
  -145.8000030517578,
  722.0)]

Cursors dont store anything, they just transfer queries to the database and get results back. 

If you just run the `cur.fetchall()` again you won't get results back unless you run the query. 

Always get back tuples, even if you only request one column.

In [18]:
cur.execute("SELECT Nest_ID FROM Bird_nests LIMIT 10")

In [19]:
cur.fetchall()


[]

In [20]:
cur.execute("SELECT Nest_ID FROM Bird_nests LIMIT 10")

# pull out the 1 value from each tuple so that it only returns a list with the first value selected
[t[0] for t in cur.fetchall()]

['14HPE1',
 '11eaba',
 '11eabaagc01',
 '11eabaagv01',
 '11eababbc02',
 '11eababsv01',
 '11eabaduh01',
 '11eabaduv01',
 '11eabarpc01',
 '11eabarpc02']

2. Get the one result, or the next result

In [21]:
cur.execute("SELECT COUNT(*) FROM Bird_nests")
cur.fetchall()

[(1547,)]

In [25]:
cur.execute("SELECT COUNT(*) FROM Bird_nests")

# only getting a single result
cur.fetchone()

(1547,)

In [24]:
cur.execute("SELECT COUNT(*) FROM Bird_nests")

# extracting the number from the mini list
cur.fetchone()[0]

1547

3. Using an iterator = but Duckdb doesnt support iterators :()

In [26]:
cur.execute("SELECT Nest_ID FROM Bird_nests LIMIT 10")
for row in cur:
    print(f"got {row[0]}")

TypeError: 'duckdb.duckdb.DuckDBPyConnection' object is not iterable

In [27]:
cur.execute("SELECT Nest_ID FROM Bird_nests LIMIT 10")
while True:
    row = cur.fetchone()
    if row == None:
        break
    # do something with row
    print(f"got nest ID {row[0]}")

got nest ID 14HPE1
got nest ID 11eaba
got nest ID 11eabaagc01
got nest ID 11eabaagv01
got nest ID 11eababbc02
got nest ID 11eababsv01
got nest ID 11eabaduh01
got nest ID 11eabaduv01
got nest ID 11eabarpc01
got nest ID 11eabarpc02


In [29]:
# """ is a way to quote multiline string

cur.execute(""" 
            CREATE TEMP TABLE temp_table AS
            SELECT * FROM Bird_nests LIMIT 10
            """)

In [31]:
cur.execute("SELECT * FROM temp_table")
cur.fetchall()

[('b14.6',
  2014,
  'chur',
  '14HPE1',
  'sepl',
  'vloverti',
  datetime.date(2014, 6, 14),
  None,
  3,
  None,
  None),
 ('b11.7',
  2011,
  'eaba',
  '11eaba',
  'wrsa',
  'bhill',
  datetime.date(2011, 7, 10),
  'searcher',
  4,
  None,
  None),
 ('b11.6',
  2011,
  'eaba',
  '11eabaagc01',
  'amgp',
  'dkessler',
  datetime.date(2011, 6, 24),
  'searcher',
  4,
  6.0,
  'float'),
 ('b11.6',
  2011,
  'eaba',
  '11eabaagv01',
  'amgp',
  'dkessler',
  datetime.date(2011, 6, 25),
  'searcher',
  3,
  3.0,
  'float'),
 ('b11.6',
  2011,
  'eaba',
  '11eababbc02',
  'bbpl',
  'dkessler',
  datetime.date(2011, 6, 24),
  'searcher',
  4,
  4.0,
  'float'),
 ('b11.7',
  2011,
  'eaba',
  '11eababsv01',
  'wrsa',
  'bhill',
  datetime.date(2011, 7, 7),
  'searcher',
  4,
  2.0,
  'float'),
 ('b11.6',
  2011,
  'eaba',
  '11eabaduh01',
  'dunl',
  'dkessler',
  datetime.date(2011, 6, 28),
  'searcher',
  3,
  2.0,
  'float'),
 ('b11.6',
  2011,
  'eaba',
  '11eabaduv01',
  'dunl',
  'dk

A note on fragility:

For example:
INSERT INTO Site VALUES ("abcd", "Foo", 35.7, -119, "?") # this order has to match column order

A less fragile way:
INSERT INTO Site (Code, Site_name, Latitude, Longitude, Something_else) 
    VALUES ("abcd", "Foo", 35.7, -119, "?")

Similarly: SELECT * is fragile

When coding, don't say SELECT *, be more explicit with coding so that you can ensure clear communication and be robust against any other changes

In [32]:
# select the specific columns
cur.execute("SELECT Site_name, Code, Latitude, Longitude FROM Site LIMIT 3")
cur.fetchall()

[('Barrow', 'barr', 71.30000305175781, -156.60000610351562),
 ('Burntpoint Creek', 'burn', 55.20000076293945, -84.30000305175781),
 ('Bylot Island', 'bylo', 73.19999694824219, -80.0)]

Extended Example:
How many nests to be have for each species?

Approach:
1. get all species
2. execute count query for each species

In [9]:
query = """
    SELECT COUNT(*) FROM Bird_nests
    WHERE Species = '%s'
"""

cur.execute("SELECT Code FROM Species LIMIT 3")
for row in cur.fetchall(): # DuckDB workaround
    code = row[0]
    prepared_query = query % code
    print(prepared_query)


    SELECT COUNT(*) FROM Bird_nests
    WHERE Species = 'agsq'


    SELECT COUNT(*) FROM Bird_nests
    WHERE Species = 'amcr'


    SELECT COUNT(*) FROM Bird_nests
    WHERE Species = 'amgp'



In [13]:
# a note on %s
s = "My name is %s"
print(s % "Greg")

s = "My name is %s and the other teachers name is %s"
print(s % ("Greg", "Julian"))

# the new way 
name = "Greg"
print(f"My name is {name}")

# the third way
print("My name is {}".format("Greg"))

My name is Greg
My name is Greg and the other teachers name is Julian
My name is Greg
My name is Greg


In [15]:
query = """
    SELECT COUNT(*) FROM Bird_nests
    WHERE Species = '%s'
"""

cur.execute("SELECT Code FROM Species LIMIT 3")
for row in cur.fetchall(): # DuckDB workaround
    code = row[0]
    prepared_query = query % code
    #print(prepared_query)
    cur2 = conn.cursor()
    cur2.execute(prepared_query)
    print(f"Species {code} has {cur2.fetchone()[0]} nests")
    cur2.close()

Species agsq has 0 nests
Species amcr has 0 nests
Species amgp has 29 nests


The above Python interpolation is dangerous an dhas caused many database hacks!

Theres a better way:

In [16]:
# change the query for Species to a '?' instead of a '%s'
query = """
    SELECT COUNT(*) FROM Bird_nests
    WHERE Species = ? 
"""

cur.execute("SELECT Code FROM Species LIMIT 3")
for row in cur.fetchall(): # DuckDB workaround
    code = row[0]
    # NOT NEEDED prepared_query = query % code
    #print(prepared_query)
    cur2 = conn.cursor()
    cur2.execute(query, [code]) # <-- added new argument here...ADD in [code] value to fill in for ?
    print(f"Species {code} has {cur2.fetchone()[0]} nests")
    cur2.close()

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Species agsq has 0 nests
Species amcr has 0 nests
Species amgp has 29 nests


Let's illustrate the danger with a different example...

Adding something else to the personell columnS

In [17]:
abbrev = "TS"
name = "Taylor Swift"
cur.execute("""
    INSERT INTO Personnel (Abbreviation, Name)
            VALUES('%s', '%s')
""" % (abbrev, name))

In [18]:
# check the personnel
cur.execute("SELECT * FROM Personnel")
cur.fetchall()[-3:]

[('emagnuson', 'Emily Magnuson'),
 ('mcorrell', 'Maureen Correll'),
 ('TS', 'Taylor Swift')]

Get a syntax error because of the apostraphe ' 

In [19]:
abbrev = "CO"
name = "Conan O'Brian" # this will cause an error in the code below
cur.execute("""
    INSERT INTO Personnel (Abbreviation, Name)
            VALUES('%s', '%s')
""" % (abbrev, name))

ParserException: Parser Error: syntax error at or near "Brian"

Now try using the ? method:

In [20]:
abbrev = "CO"
name = "Conan O'Brian" # this will cause an error in the code below
cur.execute("""
    INSERT INTO Personnel (Abbreviation, Name)
            VALUES(?,?)
""",
 [abbrev, name])

In [21]:
# check the personnel
cur.execute("SELECT * FROM Personnel")
cur.fetchall()[-3:]

[('mcorrell', 'Maureen Correll'),
 ('TS', 'Taylor Swift'),
 ('CO', "Conan O'Brian")]